In [157]:
import pandas as pd
import sklearn as sk 
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
# Выборки уже поделены на отдельные таблицы, поэтому метод сплит не понадобится 

X_train = pd.read_csv('dsc_salary_2025_train.csv')
X_test = pd.read_csv('dsc_salary_2025_test.csv')

# Сразу удаляю ненужные и шумовые колонки признаков для обоих выборок
to_drop = ['Unnamed: 0', 'id', 'first_name','noise_feature_5', 'noise_feature_1', 'noise_feature_2', 'noise_feature_3', 'noise_feature_4']

X_train.drop(to_drop, axis=1, inplace=True) 
                                # удаляю именно по оси 1, то есть ненужные столбцы
X_test.drop(to_drop, axis=1,inplace=True)

In [158]:
y = X_train['salary_class'] # ответы для обучающей выборки
X_train.drop('salary_class', axis=1,inplace=True) # удаляю таргет ответы из выборки, т.к. веса настраиваются не по ним

In [159]:
print(X_train.isnull().sum(), '\n\n')
print(X_test.isnull().sum())

hackathons_count                      0
courses_count                      1515
pet_projects_count                    0
leetcode_solved                       0
hours_study_per_week                  0
months_with_mentor                    0
gpa                                1011
internship                            0
is_mentor                             0
english_level                      3161
specialization                        0
communication_score                1961
teamworks_score                    1961
motivation_score                      0
consistency_score                     0
head_of_department                    0
hackathons_winner                     0
events_attended                       0
was_speaker                           0
bot_month                             0
coffee_cups_per_day                   0
github_commits_last_year              0
years_of_programming_experience       0
hours_sleep_per_day                   0
stress_score                          0


In [160]:
# Делю признаки на числовые и категориальные
num_feats = X_train.select_dtypes(include=[int, float]).columns.tolist()
cat_feats = X_train.select_dtypes(include=['object', 'category']).columns.tolist()


print(f"Числовые колонки: {num_feats}")
print(f"Категориальные колонки: {cat_feats}")

Числовые колонки: ['hackathons_count', 'courses_count', 'pet_projects_count', 'leetcode_solved', 'hours_study_per_week', 'months_with_mentor', 'gpa', 'communication_score', 'teamworks_score', 'motivation_score', 'consistency_score', 'events_attended', 'coffee_cups_per_day', 'github_commits_last_year', 'years_of_programming_experience', 'hours_sleep_per_day', 'stress_score', 'job_search_duration_months', 'ml_theory_score', 'coding_skill', 'math_score']
Категориальные колонки: ['internship', 'is_mentor', 'english_level', 'specialization', 'head_of_department', 'hackathons_winner', 'was_speaker', 'bot_month', 'attends_club_meetups_regularly', 'takes_part_in_memes', 'takes_part_in_informal_meetings']


In [161]:
# Заполняю пропуски средним по каждому столбцу среди числовых признаков с пропусками
for feat in num_feats:
    if X_train[feat].isnull().any():
        print(f'признак {feat}:\n')
        mean = X_train[feat].mean()
        X_train[feat] = X_train[feat].fillna(mean)
        print(f'в train заполнен значением {mean}')
        
    if X_test[feat].isnull().any():
        mean = X_test[feat].mean()
        X_test[feat] = X_test[feat].fillna(mean)
        print(f'в test заполнен значением {mean}\n')

признак courses_count:

в train заполнен значением 8.016914049016224
в test заполнен значением 7.953532608695652

признак gpa:

в train заполнен значением 3.527942481234107
в test заполнен значением 3.508271855554688

признак communication_score:

в train заполнен значением 5.009136638874625
в test заполнен значением 4.951807290365945

признак teamworks_score:

в train заполнен значением 5.016704774286869
в test заполнен значением 4.983209446608549



In [162]:
# Заполненяю пропуски модой по каждому столбцу среди категориальных признаков с пропусками
for feat in cat_feats:
    if X_train[feat].isnull().any():
        mode_val = X_train[feat].mode()[0] if not X_train[feat].mode().empty else 'Unknown'
        X_train[feat] = X_train[feat].fillna(mode_val)
        print(f"в train признак '{feat}' заполнен модой {mode_val}")

    if X_test[feat].isnull().any():
        mode_val = X_test[feat].mode()[0] if not X_test[feat].mode().empty else 'Unknown'
        X_test[feat] = X_test[feat].fillna(mode_val)
        print(f"в test признак '{feat}' заполнен модой {mode_val}")

в train признак 'english_level' заполнен модой B1
в test признак 'english_level' заполнен модой B1


In [163]:
# теперь пропусков нигде нет

print(X_train.isnull().sum(), '\n\n')

print(X_test.isnull().sum())

hackathons_count                   0
courses_count                      0
pet_projects_count                 0
leetcode_solved                    0
hours_study_per_week               0
months_with_mentor                 0
gpa                                0
internship                         0
is_mentor                          0
english_level                      0
specialization                     0
communication_score                0
teamworks_score                    0
motivation_score                   0
consistency_score                  0
head_of_department                 0
hackathons_winner                  0
events_attended                    0
was_speaker                        0
bot_month                          0
coffee_cups_per_day                0
github_commits_last_year           0
years_of_programming_experience    0
hours_sleep_per_day                0
stress_score                       0
job_search_duration_months         0
attends_club_meetups_regularly     0
t

In [164]:
# Кодируем train и test отдельно
X_train = pd.get_dummies(X_train, columns=cat_feats, dtype=int)
X_test = pd.get_dummies(X_test, columns=cat_feats, dtype=int)

In [165]:
print(X_train.describe())


       hackathons_count  courses_count  pet_projects_count  leetcode_solved  \
count      16000.000000   16000.000000        16000.000000     16000.000000   
mean           1.505625       8.016914            1.225625        41.625813   
std            1.388601       3.420440            1.338359        33.622075   
min            0.000000       0.000000            0.000000         0.000000   
25%            0.000000       6.000000            0.000000        17.000000   
50%            1.000000       8.000000            1.000000        34.000000   
75%            2.000000      10.000000            2.000000        58.000000   
max           10.000000      20.000000           10.000000       300.000000   

       hours_study_per_week  months_with_mentor           gpa  \
count          16000.000000        16000.000000  16000.000000   
mean              25.019939            4.330625      3.527942   
std                8.896910            3.399288      0.562046   
min                0.000000 

In [166]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# import pandas as pd



# plt.figure(figsize=(12, 8))
# sns.boxplot(x=X_train['leetcode_solved'])
# plt.title('Box Plot для определения выбросов')
# plt.show()

In [ ]:
# теперь устраняю выбросы


# определяю функцию межквартильного разамаха для всех числовых признаков
def igr(num_feat:str):
    Q1 = X_train[num_feat].quantile(0.25) # расчет 25-го процентиля в столбце, при этом сначала столбец монотонно упорядочивается на время расчета
    Q3 = X_train[num_feat].quantile(0.75)

    IQR = Q3 - Q1 # размах

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # print(f"Признак {num_feat}")
    # print(f"Нижняя граница для {num_feat}: {lower_bound:.2f}")
    # print(f"Верхняя граница для {num_feat}: {upper_bound:.2f}")
    
    outliers_lower = X_train[X_train[num_feat] < lower_bound]
    count_lower = len(outliers_lower)
    
    outliers_upper = X_train[X_train[num_feat] > upper_bound]
    count_upper = len(outliers_upper)
    
    total_outliers = count_lower + count_upper
    
    # print(f"\nКоличество выбросов снизу: {count_lower}")
    # print(f"Количество выбросов сверху: {count_upper}")
    # print(f"Общее количество выбросов: {total_outliers} для {num_feat}")
    # print(f"Процент выбросов от общего числа для {num_feat}: {(total_outliers / len(X_train) * 100):.2f}%\n\n\n")

    return round(total_outliers / len(X_train) * 100, 2), lower_bound, upper_bound

for feat in num_feats:
    outliers, lower, upper = igr(feat) # outliers -- процент выбросов для признака
    
    if outliers < 1:
        continue
    
    if 1 <= outliers < 5: 
        X_train[feat] = X_train[feat].clip(lower, upper) # замена на min или max допустимое значение
        # print(outliers)
        replaced_outliers = igr(feat)
        # print(replaced_outliers)

    elif 5 <= outliers < 15:
        median_val = X_train[feat].median() 
        
        outlier_mask = (X_train[feat] < lower) | (X_train[feat] > upper)

        X_train.loc[outlier_mask, feat] = median_val
        # print(f"  Заменено {outlier_mask.sum()} значений для {feat} на {median_val:.2f}")
    
# for feat in num_feats:
#     outliers, lower, upper = igr(feat)
#     print(outliers)

1.08
(0.0, np.float64(-3.0), np.float64(5.0))
3.21
(0.0, np.float64(-44.5), np.float64(119.5))
  Заменено 2049 значений для coffee_cups_per_day на 3.00
3.17
(0.0, np.float64(-42.5), np.float64(113.5))
1.14
(0.0, np.float64(-4.5), np.float64(15.5))
0.0
0.88
0.92
0.0
0.74
0.04
0.0
0.0
0.0
0.0
0.42
0.4
0.0
0.0
0.59
0.81
0.0
0.0
0.0
0.0
0.0


In [172]:
model = SVC()

model.fit(X_train, y) # обучаю моделью опорных векторов 

y_pred = model.predict(X_test)

test_original = pd.read_csv('dsc_salary_2025_test.csv')
ids = test_original['id']

results = pd.DataFrame({
    'id': ids,
    'pred_value': y_pred
})

results.to_csv('predicted_test.csv', index=False)